In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from glob import glob
from os.path import join, basename, splitext
from tqdm import tqdm

import shutil



In [ ]:
target_root = '../../data/tum_material'
os.path.isdir(target_root)

In [ ]:
from matplotlib.colors import ListedColormap

# material classes
color_mapping = {
    (255, 0, 255): "Reinforced Concrete",
    (255, 135, 0): "Unreinforced Concrete",
    (0, 255, 36): "Precast Concrete",
    (25, 0, 255): "Masonry",
    (25, 124, 13): "Slit",
    (255, 250, 0): "XPS Insulation",
    (255, 0, 0): "Hard Insulation",
    (0, 235, 248): "Soft Insulation (Computer-Aided)",
    (147, 27, 236): "Soft Insulation (Hand-Drawn)",
    (0, 0, 0): "Non-Material"
}

color_to_class = {
    (255, 0, 249): 1,
    (255, 135, 0): 2,
    (0, 255, 36): 3,
    (25, 0, 255): 4,
    (25, 124, 13): 5,
    (255, 249, 0): 6,
    (255, 0, 0): 7,
    (0, 238, 255): 8,
    (147, 27, 236): 8,
    (0, 0, 0): 0
}

class_to_labels = {
    0: "Non-Material",
    1: "Reinforced Concrete",
    2: "Unreinforced Concrete",
    3: "Precast Concrete",
    4: "Masonry",
    5: "Slit",
    6: "XPS Insulation",
    7: "Hard Insulation",
    8: "Soft Insulation ",
}

# Get a single color for each class
# If a class has multiple RGBs (like class 8), just pick one
class_colors = {}
for rgb, class_id in color_to_class.items():
    if class_id not in class_colors:
        class_colors[class_id] = rgb

# Sort by class index
sorted_classes = sorted(class_colors.items())

# Normalize RGBs to [0,1] range
cmap_colors = [np.array(rgb) / 255.0 for _, rgb in sorted_classes]

# Create ListedColormap
cmap_material = ListedColormap(cmap_colors)
label_names = [class_to_labels[class_id] for class_id, _ in sorted_classes]



In [ ]:
masks_dirs = glob(join(target_root, '*', 'masks'))
# Define a tolerance value (e.g., ±10 per channel)

masks_dirs

In [ ]:
masks_dirs = glob(join(target_root, '*', 'masks'))
# Define a tolerance value (e.g., ±10 per channel)
COLOR_TOLERANCE = 10
for masks_dir in masks_dirs:
    print(masks_dir)
    anno_dir = join(os.path.dirname(masks_dir), 'anno')
    if not os.path.exists(anno_dir):
        os.makedirs(anno_dir)
    mask_paths = glob(join(masks_dir, '*.png'))
    for mask_path in tqdm(mask_paths):
        mask_img = Image.open(mask_path).convert('RGB')
        mask_np = np.asarray(mask_img)
        
        anno_img = np.zeros((mask_np.shape[0], mask_np.shape[1]), dtype=np.uint8)

        # map each color to class
        for color, class_id in color_to_class.items():
            diff = np.abs(mask_np - np.array(color))
            matches = np.all(diff <= COLOR_TOLERANCE, axis=-1)
            anno_img[matches] = class_id

        # save single-channel label image
        output_filename = os.path.basename(mask_path)
        output_path = join(anno_dir, output_filename)
        Image.fromarray(anno_img).save(output_path)




In [ ]:
# check the results
i = 0
anno_dirs = sorted(glob(join(target_root, '*', 'anno')))
image_dirs = sorted(glob(join(target_root, '*', 'images')))
anno_dir = anno_dirs[i]
image_dir = image_dirs[i]

anno_images = sorted(glob(join(anno_dir, '*.png')))
drawing_imgs = sorted(glob(join(image_dir, '*.jpg')))

for rand_i in np.random.choice(range(len(anno_images)), 10, replace=False):
    assert splitext(basename(drawing_imgs[rand_i]))[0] == splitext(basename(anno_images[rand_i]))[0]
    drawing = Image.open(drawing_imgs[rand_i])
    drawing_np = np.asarray(drawing)
    anno = Image.open(anno_images[rand_i])
    anno_np = np.asarray(anno)
    fig, axs = plt.subplots(1, 2, figsize=(10,5), dpi=150)
    axs[0].imshow(drawing_np)
    axs[0].set_title('drawing')
    # axs[1].matshow(anno_np, vmin=0, vmax=8, cmap='jet')
    axs[1].imshow(anno_np, vmin=0, vmax=8, cmap=cmap_material)
    axs[1].set_title('labels')
    plt.tight_layout()

In [ ]:
# notes
# the tile size appear to be like 650px

In [ ]:
len(anno_images)

In [ ]:
len(drawing_imgs)